В этом ноубуке используются обeченные тестовые модели, с их помощью происходит сэмплирование и далее закодированные категориальные переменные переводятся в изначальные. После чего замеряется качество.

Способ нормализации: quantile transformation для числовых признаков и standart scale для категориальных OHE

Добавление шума: ко всем признакам

In [1]:
%cd ..

/workspace/TabDDPM_copy/TabDDPM


/venv/main/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import json
import pandas as pd
import torch
import matplotlib.pyplot as plt

from utils import *
from models.tabddpm_ON_QnSC.tabddpm_ON_QnSC import *

from tqdm.notebook import tqdm

from eval.base_metrics import calculate_base_metrics
from eval.similarity import calculate_similarity
from eval.mle import calculate_mle
from eval.alpha_beta import calculate_alpha_beta
from eval.detection import calculate_detection
from eval.dcr import calculate_DCR

### Подготовка

In [3]:
dataname = 'magic_ON_QnSC_200k'
dataname_initial = 'magic'
model_short = 'tabddpm_ON_QnSC'
model_name = 'tabddpm_ON_QnSC'
model_name_postfix = '_200k'

idx = list(range(10))
constants = [0, 0.001, 0.01, 0.1, 0.25, 0.5]

CONFIG.add_arg('save_cat', None)
target_cat=True

In [4]:
if torch.cuda.is_available():
    device = f'cuda'
else:
    device = 'cpu'
CONFIG.add_arg('device', device)
print(f"Device: {device}")

CONFIG.add_arg('dataname', dataname)
CONFIG.add_arg('real_path',
                   f'./synthetic/{dataname}/initial_real.csv')
CONFIG.add_arg('test_path',
                   f'./synthetic/{dataname}/initial_test.csv')

CONFIG.add_arg('info_path',
                   f'./data/{dataname}/initial_info.json')

Device: cuda


In [5]:
model_save_path_hist = []
for sigma in constants:
    model_save_path=f"./models/{model_name}/ckpt/{dataname_initial}/{model_name}{model_name_postfix}_const_{str(sigma).replace('.', '_')}"
    model_save_path_hist.append(model_save_path)

for i in idx:
    model_save_path=f"./models/{model_name}/ckpt/{dataname_initial}/{model_name}{model_name_postfix}_mult_{i}"
    model_save_path_hist.append(model_save_path)

In [6]:
from pickle import dump, load
with open(f'data/{dataname}/normalizers.json', 'r') as f:
        normalizers = json.load(f)
normalizers['num_normalizer'] = load(open(f'./data/{dataname}/num_normalizer_{dataname}.pkl', 'rb'))
normalizers['cat_normalizer'] = load(open(f'./data/{dataname}/cat_normalizer_{dataname}.pkl', 'rb'))
print(f"normalizers received from `data/{dataname}` folder")

normalizers received from `data/magic_ON_QnSC_200k` folder


In [7]:
CONFIG.get_all_args()

{'dataname': 'magic_ON_QnSC_200k',
 'method': None,
 'device': 'cuda',
 'mode': 'train',
 'train': 1,
 'sample_save_path': None,
 'sigma_scheduller_name': 'constant',
 'sigma_value': 0.001,
 'num_noise': 1,
 'real_path': './synthetic/magic_ON_QnSC_200k/initial_real.csv',
 'test_path': './synthetic/magic_ON_QnSC_200k/initial_test.csv',
 'info_path': './data/magic_ON_QnSC_200k/initial_info.json',
 'save_path': None,
 'num_clusters': 25,
 'save_cat': None}

### Sample + Eval

In [8]:
overall_metrics = {}

for model_save_path in model_save_path_hist:
    model_name = model_save_path.split('/')[-1]
    sample_save_path = f'./synthetic/{dataname}/{model_name}.csv'

    CONFIG.add_arg('method', model_name)
    CONFIG.add_arg('sample_save_path',
                       f"synthetic/{CONFIG.get_arg('dataname')}/{CONFIG.get_arg('method')}.csv")

    overall_metrics[model_name] = {}

    print(model_save_path, model_name, sample_save_path)

    tabddpm_noise_ohe = TabDDPM_OHE_Noise_QnSC(CONFIG, model_save_path=model_save_path, sigmas=None)
    tabddpm_noise_ohe.sample(sample_save_path=sample_save_path)
    postsample_OHE(dataname, f'./synthetic/{dataname}/initial_{model_name}.csv', normalizers=normalizers, target_cat=target_cat)

    # подсчет метрик
    try:
        overall_metrics[model_name]['base_metrics'] = calculate_base_metrics(make_binary=False)
    except Exception as e:
        overall_metrics[model_name]['base_metrics'] = np.nan
    overall_metrics[model_name]['similarity'] = calculate_similarity()
    overall_metrics[model_name]['mle'] = calculate_mle()
    overall_metrics[model_name]['detection'] = calculate_detection()
    overall_metrics[model_name]['DCR'] = calculate_DCR()
    overall_metrics[model_name]['quality'] = calculate_alpha_beta()

./models/tabddpm_ON_QnSC/ckpt/magic/tabddpm_ON_QnSC_200k_const_0 tabddpm_ON_QnSC_200k_const_0 ./synthetic/magic_ON_QnSC_200k/tabddpm_ON_QnSC_200k_const_0.csv
No NaNs in numerical features, skipping
mlp
Sample timestep    0
Sample timestep    0
Shape torch.Size([17118, 11])
Sampling time: 93.52240109443665
Sample path changed to ./synthetic/magic_ON_QnSC_200k/initial_tabddpm_ON_QnSC_200k_const_0.csv
TABDDPM_ON_QNSC_200K_CONST_0: MAGIC_ON_QNSC_200K Dataset
Original Logistic: 0.833
Synthetic Logistic: 0.830
Original Tree: 0.878
Synthetic Tree: 0.872 

TABDDPM_ON_QNSC_200K_CONST_0 Accuracy Loss: MAGIC_ON_QNSC_200K
Logistic: 0.401%
Tree: 0.679% 

CALCULATING COLUMN-WISE DENSITY ESTIMATION & PAIR-WISE COLUMN CORRELATION...
Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 11/11 [00:00<00:00, 101.61it/s]|
Column Shapes Score: 98.96%

(2/2) Evaluating Column Pair Trends: |██████████| 55/55 [00:00<00:00, 94.93it/s]|
Column Pair Trends Score: 97.49%

Overall Score (Average): 98

DONE! 

DRAW COLUMN VALUES DISTRIBUTIONS...


DONE!


100%|██████████| 36/36 [00:07<00:00,  4.66it/s]


Saving scores to  eval/mle/magic_ON_QnSC_200k/tabddpm_ON_QnSC_200k_const_0.json
ROC - AUC обучения на синтетических данных TABDDPM_ON_QNSC_200K_CONST_0: 0.946 ± 0.008
magic_ON_QnSC_200k, tabddpm_ON_QnSC_200k_const_0: 0.9796035657241029
n_clusters: 25
DCR Score, a value closer to 0.5 is better
magic_ON_QnSC_200k-tabddpm_ON_QnSC_200k_const_0, DCR Score = 0.5541535226077813
=========== All Features ===========
Data shape:  (17118, 12)
alpha precision: 0.975052, beta recall: 0.643907
./models/tabddpm_ON_QnSC/ckpt/magic/tabddpm_ON_QnSC_200k_const_0_001 tabddpm_ON_QnSC_200k_const_0_001 ./synthetic/magic_ON_QnSC_200k/tabddpm_ON_QnSC_200k_const_0_001.csv
No NaNs in numerical features, skipping
mlp
Sample timestep    0
Sample timestep    0
Shape torch.Size([17118, 11])
Sampling time: 96.59213185310364
Sample path changed to ./synthetic/magic_ON_QnSC_200k/initial_tabddpm_ON_QnSC_200k_const_0_001.csv
TABDDPM_ON_QNSC_200K_CONST_0_001: MAGIC_ON_QNSC_200K Dataset
Original Logistic: 0.833
Synthetic L

...

Saving scores to  eval/mle/magic_ON_QnSC_200k/tabddpm_ON_QnSC_200k_mult_8.json
ROC - AUC обучения на синтетических данных TABDDPM_ON_QNSC_200K_MULT_8: 0.922 ± 0.007
magic_ON_QnSC_200k, tabddpm_ON_QnSC_200k_mult_8: 1.0
n_clusters: 25
DCR Score, a value closer to 0.5 is better
magic_ON_QnSC_200k-tabddpm_ON_QnSC_200k_mult_8, DCR Score = 0.5202710597032364
=========== All Features ===========
Data shape:  (17118, 12)
alpha precision: 0.990740, beta recall: 0.496316
./models/tabddpm_ON_QnSC/ckpt/magic/tabddpm_ON_QnSC_200k_mult_9 tabddpm_ON_QnSC_200k_mult_9 ./synthetic/magic_ON_QnSC_200k/tabddpm_ON_QnSC_200k_mult_9.csv
No NaNs in numerical features, skipping
mlp
Sample timestep    0
Sample timestep    0
Shape torch.Size([17118, 11])
Sampling time: 95.16340065002441
Sample path changed to ./synthetic/magic_ON_QnSC_200k/initial_tabddpm_ON_QnSC_200k_mult_9.csv
TABDDPM_ON_QNSC_200K_MULT_9: MAGIC_ON_QNSC_200K Dataset
Original Logistic: 0.833
Synthetic Logistic: 0.832
Original Tree: 0.872
Syntheti

DONE! 

DRAW COLUMN VALUES DISTRIBUTIONS...


DONE!


100%|██████████| 36/36 [00:07<00:00,  5.04it/s]


Saving scores to  eval/mle/magic_ON_QnSC_200k/tabddpm_ON_QnSC_200k_mult_9.json
ROC - AUC обучения на синтетических данных TABDDPM_ON_QNSC_200K_MULT_9: 0.934 ± 0.005
magic_ON_QnSC_200k, tabddpm_ON_QnSC_200k_mult_9: 0.9979936587453607
n_clusters: 25
DCR Score, a value closer to 0.5 is better
magic_ON_QnSC_200k-tabddpm_ON_QnSC_200k_mult_9, DCR Score = 0.5209720761771235
=========== All Features ===========
Data shape:  (17118, 12)
alpha precision: 0.993891, beta recall: 0.516112


In [9]:
final_metrics_table = []
for m in overall_metrics.keys():
    # Сбор таблички результатов
    tmp = pd.DataFrame([{'Model':model_short, 'Type':m, 'Data':dataname_initial}])
    tmp.columns = pd.MultiIndex.from_tuples([('', i) for i in tmp.columns])
    result = [tmp]
    
    for metric_group in overall_metrics[m].keys():
        tmp = pd.DataFrame([overall_metrics[m][metric_group]])
        tmp.columns = pd.MultiIndex.from_tuples([(metric_group, i) for i in tmp.columns])
        result.append(tmp)
    result = pd.concat(result, axis = 1)
    final_metrics_table.append(result)
    
final_metrics_table = pd.concat(final_metrics_table)

In [11]:
pd.set_option('display.max_columns', None)
final_metrics_table.round(4).sort_values([('similarity', 'Overall Score (Average), %')],
                                                                          ascending=False)

base_metrics  \
             Model                              Type   Data Original Logistic   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_7  magic            0.8331   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_2  magic            0.8331   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_1  magic            0.8331   
0  tabddpm_ON_QnSC    tabddpm_ON_QnSC_200k_const_0_1  magic            0.8331   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_5  magic            0.8331   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_8  magic            0.8331   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_0  magic            0.8331   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_3  magic            0.8331   
0  tabddpm_ON_QnSC  tabddpm_ON_QnSC_200k_const_0_001  magic            0.8331   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_9  magic            0.8331   
0  tabddpm_ON_QnSC   tabddpm_ON_QnSC_200k_const_0_25  magic            0.8331   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_6  magic            0.8331   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_4  magic            0.8331   
0  tabddpm_ON_QnSC      tabddpm_ON_QnSC_200k_const_0  magic            0.8331   
0  tabddpm_ON_QnSC   tabddpm_ON_QnSC_200k_const_0_01  magic            0.8331   
0  tabddpm_ON_QnSC    tabddpm_ON_QnSC_200k_const_0_5  magic            0.8331   

                                                                             \
  Synthetic Logistic Original Tree Synthetic Tree Accuracy Loss Logistic, %   
0             0.8327        0.8720         0.8523                    0.0419   
0             0.8342        0.8720         0.8683                   -0.1333   
0             0.8301        0.8720         0.8629                    0.3499   
0             0.8315        0.8720         0.8718                    0.1916   
0             0.8326        0.8720         0.8590                    0.0501   
0             0.8316        0.8720         0.8702                    0.1747   
0             0.8331        0.8720         0.8605                   -0.0083   
0             0.8277        0.8720         0.8607                    0.6428   
0             0.8311        0.8720         0.8504                    0.2344   
0             0.8316        0.8720         0.8612                    0.1758   
0             0.8318        0.8720         0.8621                    0.1482   
0             0.8312        0.8720         0.8775                    0.2253   
0             0.8320        0.8720         0.8656                    0.1252   
0             0.8297        0.8778         0.8718                    0.4010   
0             0.8302        0.8720         0.8598                    0.3443   
0             0.8352        0.8720         0.8433                   -0.2531   

                                    similarity                              \
  Accuracy Loss Tree, % Column Shapes Score, % Column Pair Trends Score, %   
0                2.2670                99.1487                     99.1581   
0                0.4322                99.2549                     98.9354   
0                1.0516                99.2746                     98.6899   
0                0.0259                99.1768                     98.6887   
0                1.4961                99.2613                     98.5472   
0                0.2163                99.2177                     98.4789   
0                1.3210                99.1158                     98.5451   
0                1.3008                99.2634                     98.3674   
0                2.4780                99.0159                     98.5007   
0                1.2431                99.1205                     98.3347   
0                1.1389                98.8375                     98.5251   
0               -0.6214                99.0547                     97.6667   
0                0.7387                99.1195                     97.5188   
0            

In [12]:
import os
if not os.path.exists(f'./eval/total/{dataname_initial}'):
    os.makedirs(f'./eval/total/{dataname_initial}')
final_metrics_table.to_csv(f'./eval/total/{dataname_initial}/{model_short}{model_name_postfix}_final_metrics_table.csv', index=False)